In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd

In [2]:
# data = pd.read_csv('data/kaggle_data/chessData.csv')
data = pd.read_csv('chessdata_100k.csv')

In [3]:
data.head()

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26


In [4]:
class ChessDataset(torch.utils.data.Dataset):
    def __init__(self, board, score):
        self.board = board
        self.score = score

    def __len__(self):
        return len(self.score)

    def __getitem__(self, index):
        # print(self.data[index])
        board, score = self.board[index], self.score[index]

        piece_val_map = {'p':-1,'k':-600,'b':-3,'r':-5,'q':-9,'n':-3,'P':1,'K':600,'B':3,'R':5,'Q':9,'N':3}

        input_data = []
        for char in board.split(" ")[0]:
            if char.isdigit():
                for _ in range(int(char)):
                    input_data.append(0)
            else:
                if char == '/':
                    continue
                input_data.append(piece_val_map[char])

        if board.split(" ")[1] == "w":
            input_data.append(1)
        else:
            input_data.append(0)

        if score.startswith("#"):
            score = score[1:]
            
        data = torch.tensor(input_data, dtype=torch.float32)
        label = torch.tensor(int(score), dtype=torch.float32)

        return data, label

In [5]:
dataset = ChessDataset(data['FEN'], data['Evaluation'])

In [6]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, pin_memory=False)

In [7]:
for data in dataloader:
    print(data[0][0])
    break

tensor([  -5.,    0.,    0.,    0.,    0.,   -5., -600.,    0.,   -1.,   -3.,
           0.,    0.,   -9.,   -1.,   -1.,   -1.,    0.,   -1.,    0.,   -1.,
          -3.,    0.,    0.,    0.,    0.,    0.,   -1.,    0.,   -1.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    1.,    1.,    0.,    0.,
           0.,    3.,    1.,    1.,    3.,    0.,    9.,    0.,    1.,    1.,
           0.,    0.,    0.,    0.,    1.,    1.,    0.,    0.,    0.,    5.,
           0.,    5.,  600.,    0.,    0.])


In [16]:
class EvalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(65, 1024)
        self.hidden_layer = nn.Linear(1024, 512)
        self.output_layer = nn.Linear(512, 1)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = nn.functional.relu(x)
        x = self.hidden_layer(x)
        x = nn.functional.relu(x)
        x = self.output_layer(x)
        return x

In [17]:
def train(model, dataloader, optimizer, criterion, epochs, device):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(dataloader):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = model(data)

            # Loss
            loss = criterion(output, target)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()

            # Gradient descent
            optimizer.step()
        
        running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} loss: {loss.item()} runnning_loss: {running_loss/len(dataloader)}")

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
model = EvalNetwork()
model.to(device)

EvalNetwork(
  (input_layer): Linear(in_features=65, out_features=1024, bias=True)
  (hidden_layer): Linear(in_features=1024, out_features=512, bias=True)
  (output_layer): Linear(in_features=512, out_features=1, bias=True)
)

In [20]:
# create the loss function
criterion = nn.MSELoss()

# create the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# num of epochs
epochs = 100

# train the model
train(model, dataloader,  optimizer, criterion,epochs, device)


c:\Users\dahiy_uokx4\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\dahiy_uokx4\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100 loss: 2470269.75 runnning_loss: 3158.9127237851662
Epoch 2/100 loss: 81602.265625 runnning_loss: 104.3507233056266
Epoch 3/100 loss: 28883.37109375 runnning_loss: 36.935257153132994
Epoch 4/100 loss: 30968.37109375 runnning_loss: 39.60149756234016
Epoch 5/100 loss: 35819.234375 runnning_loss: 45.80464753836317
Epoch 6/100 loss: 1998291.0 runnning_loss: 2555.359335038363
Epoch 7/100 loss: 179781.9375 runnning_loss: 229.90017583120203
Epoch 8/100 loss: 109786.828125 runnning_loss: 140.39236333120203
Epoch 9/100 loss: 353121.0 runnning_loss: 451.5613810741688
Epoch 10/100 loss: 18873.13671875 runnning_loss: 24.134445931905372
Epoch 11/100 loss: 433084.6875 runnning_loss: 553.8167359335039
Epoch 12/100 loss: 1172385.75 runnning_loss: 1499.2145140664961
Epoch 13/100 loss: 81422.59375 runnning_loss: 104.12096387468031
Epoch 14/100 loss: 79808.703125 runnning_loss: 102.05716512148338
Epoch 15/100 loss: 130241.5625 runnning_loss: 166.54931265984655
Epoch 16/100 loss: 461893.125 run

In [21]:
with torch.no_grad():
    y_eval = model.forward(torch.tensor([0.,0.,0.,0.,-5.,   -3., -600.,    0.,    0.,    0.,
          -1.,   -9.,    0.,   -1.,   -1.,    0.,   -1.,    0.,   -3.,    0.,
          -5.,    0.,    0.,   -1.,    0.,   -1.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    1.,    0.,    0.,    3.,   -1.,    0.,    0.,
           1.,    0.,    1.,    1.,    0.,    3.,    0.,    0.,    0.,    0.,
           9.,    0.,    0.,    1.,    1.,    1.,    5.,    0.,    0.,    5.,
           0.,    0.,  600.,    0.,    0.], dtype=torch.float32).to(device))

In [22]:
y_eval

tensor([29.9571], device='cuda:0')

In [23]:
torch.save(model.state_dict(), "model/nn_1024_512.pt")

In [24]:
model

EvalNetwork(
  (input_layer): Linear(in_features=65, out_features=1024, bias=True)
  (hidden_layer): Linear(in_features=1024, out_features=512, bias=True)
  (output_layer): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
model.output_layer.weight.shape